# Connectivity Status Assessment and Action Plan {-}

The planning team devised two Key Ecological Attributes (KEAs) and associated indicators to assess the current connectivity status of the watershed – Accessible Habitat and Accessible Overwintering Habitat (@tbl-connectivity). KEAs are the key aspects of anadromous salmon ecology that are being targeted by this WCRP. The connectivity status of Anadromous Salmon was used to establish goals to improve habitat connectivity in the watershed and will be the baseline against which progress is tracked over time. 

The current connectivity status assessment relies on GIS analyses to map known and modelled barriers to fish passage, identify stream reaches that have potential spawning and rearing habitat, estimate the proportion of habitat that is currently accessible to target species, and prioritize barriers for field assessment that would provide the greatest gains in connectivity. To support a flexible prioritization framework to identify priority barriers in the watershed, two assumptions are made: 1,any modelled (i.e., passability status is unknown) or partial barriers are treated as complete barriers to passage and 2, the habitat modelling is binary, it does not assign any habitat quality values. As such, the current connectivity status will be refined over time as more data on habitat and barriers are collected. For more detail on how the connectivity status assessments were conducted, see Appendix B. 


In [1]:
#| echo: false
#| warning: false

import requests
import json
import pandas

def barrier_extent(barrier_type):

    request = 'http://159.89.114.239:9002/functions/postgisftw.wcrp_barrier_extent/items.json?watershed_group_code=HORS&barrier_type=' + barrier_type

    response_api = requests.get(request)
    parse = response_api.text
    result = json.loads(parse)

    blocked_km = result[0]['all_habitat_blocked_km']
    blocked_pct = result[0]['extent_pct']

    return blocked_km, blocked_pct

def barrier_count(barrier_type):
    request = 'http://159.89.114.239:9002/functions/postgisftw.wcrp_barrier_count/items.json?watershed_group_code=HORS&barrier_type=' + barrier_type

    response_api = requests.get(request)
    parse = response_api.text
    result = json.loads(parse)

    n_passable = result[0]['n_passable']
    n_barrier = result[0]['n_barrier']
    n_potential = result[0]['n_potential']
    n_unknown = result[0]['n_unknown']

    sum_bar = (n_passable, n_barrier, n_potential, n_unknown)

    return n_passable, n_barrier, n_potential, n_unknown, sum(sum_bar)

def barrier_severity(barrier_type):

    request = 'http://159.89.114.239:9002/functions/postgisftw.wcrp_barrier_severity/items.json?watershed_group_code=HORS&barrier_type=' + barrier_type

    response_api = requests.get(request)
    parse = response_api.text
    result = json.loads(parse)

    n_assessed_barrier = result[0]['n_assessed_barrier']
    n_assess_total = result[0]['n_assess_total']
    pct_assessed_barrier = result[0]['pct_assessed_barrier']

    return n_assessed_barrier, n_assess_total, pct_assessed_barrier

def watershed_connectivity(habitat_type):

    request = 'http://159.89.114.239:9002/functions/postgisftw.wcrp_watershed_connectivity_status/items.json?watershed_group_code=HORS&barrier_type=' + habitat_type

    response_api = requests.get(request)
    parse = response_api.text
    result = json.loads(parse)

    connect_stat = result[0]['connectivity_status']

    return str(round(connect_stat))

num_dam = barrier_severity('DAM')[1]
km_dam = barrier_extent('DAM')[0]
pct_dam = barrier_extent('DAM')[1]
resource_km = barrier_extent('ROAD, RESOURCE/OTHER')[0]
resource_pct = round(barrier_extent('ROAD, RESOURCE/OTHER')[1])
demo_km = barrier_extent('ROAD, DEMOGRAPHIC')[0]
demo_pct = round(barrier_extent('ROAD, DEMOGRAPHIC')[1])
resource_sev = round(barrier_severity('ROAD, RESOURCE/OTHER')[2])
demo_sev = round(barrier_severity('ROAD, DEMOGRAPHIC')[2])
sum_road = barrier_severity('ROAD, RESOURCE/OTHER')[1] + barrier_severity('ROAD, DEMOGRAPHIC')[1]

In [2]:
#| label: tbl-connectivity
#| tbl-cap: Connectivity status assessment for (a) linear habitat (spawning and rearing) and (b) overwintering habitat in the Horsefly River watershed. The Available Habitat KEA is evaluated by dividing the length of linear habitat that is currently accessible to target species by the total length of all linear habitat in the watershed. The Available Overwintering Habitat KEA is evaluated as the sum of all areal overwintering habitat that is accessible to target species.
#| warning: false
#| echo: false

import pandas as pd
import numpy as np
import matplotlib as mpl
import warnings

warnings.filterwarnings('ignore')

df = pd.DataFrame({"Target Species":["Andromous Salmon"," "],
                   "KEA":["Available Habitat"," "],
                   "Indicator":["% of total linear habitat","Current Status:"],
                   "Poor":["<80%"," "],
                   "Fair":["  "," "],
                   "Good":["81-90%"," "],
                   "Very Good":[">90%", watershed_connectivity("ALL")]
                   })


def highlighttab7(val):
    red = '#ff0000;'
    yellow = '#ffff00;'
    lgreen = '#92d050;'
    dgreen = '#03853e;'

    if val=="<80%" : color = red
    elif val[0:].isdigit() and int(val) < 80 : color = red
    elif val=="  ": color = yellow
    elif val=="81-90%"  : color = lgreen
    elif val[0:].isdigit() and (int(val) >= 80 and int(val) < 90) : color = lgreen 
    elif val ==">90%": color = dgreen
    elif val[0:].isdigit() and int(val) >= 90 : color = dgreen 
    elif val == "Current Status:" : return "font-weight: bold"
    else: color = 'white'
    return 'background-color: %s' % color

df.style.applymap(highlighttab7).set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }]).hide_index()

Target Species,KEA,Indicator,Poor,Fair,Good,Very Good
Andromous Salmon,Available Habitat,% of total linear habitat,<80%,,81-90%,>90%
,,Current Status:,,,,91


**Comments:** Indicator rating definitions are based on the consensus decisions of the planning team, including the decision not to define Fair. The current status is based on the CWF Barrier Prioritization Model output, which is current as of March 2022.


In [3]:
#| warning: false
#| echo: false

#creating table 7
import pandas as pd
import numpy as np
import matplotlib as mpl


warnings.filterwarnings('ignore')

df = pd.DataFrame({"Target Species":["Andromous Salmon",""],
                   "KEA":["Available Overwintering Habitat",""],
                   "Indicator":["Total Area (m2) of overwintering habitat accessible","Current Status:"],
                   "Poor":["?",""],
                   "Fair":[" ?",""],
                   "Good":["? ",""],
                   "Very Good":[" ? ",""]
                   })

def highlighttab7b(val):
    red = '#ff0000;'
    yellow = '#ffff00;'
    lgreen = '#92d050;'
    dgreen = '#03853e;'

    if val=="?" : color = red
    elif val[0:].isdigit() and int(val) < 80 : color = red
    elif val==" ?": color = yellow
    elif val=="? "  : color = lgreen
    elif val[0:].isdigit() and (int(val) >= 80 and int(val) < 90) : color = lgreen 
    elif val ==" ? ": color = dgreen
    elif val[0:].isdigit() and int(val) >= 90 : color = dgreen 
    elif val == "Current Status:" : return "font-weight: bold"
    else: color = 'white'
    return 'background-color: %s' % color

df.style.applymap(highlighttab7b).set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }]).hide_index()


Target Species,KEA,Indicator,Poor,Fair,Good,Very Good
Andromous Salmon,Available Overwintering Habitat,Total Area (m2) of overwintering habitat accessible,?,?,?,?
,,Current Status:,,,,


**Comments:** No baseline data exists on the extent of overwintering habitat in the watershed. A priority action is included in the Operational Plan (strategy 2.3) to develop a habitat layer, and this will be used to inform this connectivity status assessment in the future.


## Barrier Types {-}

The following table highlights which barrier types pose the greatest threat to anadromous salmon in the watershed. The results of this assessment were used to inform the subsequent planning steps, as well as to identify knowledge gaps where there is little spatial data to inform the assessment for a specific barrier type. 


In [4]:
#| label: tbl-barriertype
#| tbl-cap: Connectivity status assessment for (a) linear habitat (spawning and rearing) and (b) overwintering habitat in the Horsefly River watershed. The Available Habitat KEA is evaluated by dividing the length of linear habitat that is currently accessible to target species by the total length of all linear habitat in the watershed. The Available Overwintering Habitat KEA is evaluated as the sum of all areal overwintering habitat that is accessible to target species.
#| warning: false
#| echo: false

from ipywidgets import *
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

#condition
def condition(pct):
    rating = ""
    if pct < 30 : rating = "Low"
    elif (pct >= 30) and (pct < 71) : rating = "Medium"
    elif (pct >= 71) and (pct < 90) : rating = "High"
    else : rating = "Very High"
    return rating

#rating classifier
def rating(threat, barrier):
    if threat == "extent":
        if barrier == "DAM":
            pct = barrier_extent(barrier)[1]
            rating = condition(pct)
        elif barrier == "ROAD":
            pct = barrier_extent('ROAD, RESOURCE/OTHER')[1] + barrier_extent('ROAD, DEMOGRAPHIC')[1]
            rating = condition(pct)
    elif threat == "severity":
        if barrier == "DAM":
            pct = barrier_severity(barrier)[2]
            rating = condition(pct)
        elif barrier == "ROAD":
            pct = barrier_severity('ROAD, RESOURCE/OTHER')[2] + barrier_severity('ROAD, DEMOGRAPHIC')[2]
            rating = condition(pct)
            
    return rating
            

        




df = pd.DataFrame({"Barrier Types":["Road-Stream Crossings","Lateral Barriers","Small Dams(<3m height)","Trail-stream Crossings", "Natural Barriers"],
                   "Extent":[rating("extent", "ROAD"),"High",rating("extent", "DAM"), "Low", "Medium"],
                   "Severity":[rating("severity", "ROAD"),"Very High",rating("severity", "DAM"), "Low", "High"],
                   "Irreversibility":["Medium","High","High", "Medium", "Low"],
                   "Overall Threat Rating:":["Very High","High","Medium", "Low", "Low"]
                   }).style.set_properties(subset=["Overall Threat Rating:"], **{'font-weight': 'bold'})

def highlight(val):
    red = '#ff0000;'
    yellow = '#ffff00;'
    lgreen = '#92d050;'
    dgreen = '#03853e;'

    if val=="Very High": color = red
    elif val=="High": color = yellow
    elif val=="Medium": color = lgreen
    elif val =="Low": color = dgreen
    else: color = 'white'
    return 'background-color: %s' % color

#df = df.style.set_properties(subset=["Overall Threat Rating"], **{'font-weight': 'bold'})

data = df.applymap(highlight).hide_index()

data.set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }])

Barrier Types,Extent,Severity,Irreversibility,Overall Threat Rating:
Road-Stream Crossings,Low,Very High,Medium,Very High
Lateral Barriers,High,Very High,High,High
Small Dams(<3m height),Low,High,High,Medium
Trail-stream Crossings,Low,Low,Medium,Low
Natural Barriers,Medium,High,Low,Low


### Small Dams (<3 m height) {-}


In [5]:
#| echo: false
#| include: true


num_dam = barrier_severity('DAM')[1]
km_dam = barrier_extent('DAM')[0]
pct_dam = barrier_extent('DAM')[1]
resource_km = barrier_extent('ROAD, RESOURCE/OTHER')[0]
resource_pct = round(barrier_extent('ROAD, RESOURCE/OTHER')[1])
demo_km = barrier_extent('ROAD, DEMOGRAPHIC')[0]
demo_pct = round(barrier_extent('ROAD, DEMOGRAPHIC')[1])
resource_sev = round(barrier_severity('ROAD, RESOURCE/OTHER')[2])
demo_sev = round(barrier_severity('ROAD, DEMOGRAPHIC')[2])
sum_road = barrier_severity('ROAD, RESOURCE/OTHER')[1] + barrier_severity('ROAD, DEMOGRAPHIC')[1]

```{r echo=FALSE}
#| echo: false
#| include: true
#| warning: false

library(reticulate)


num_dam <- py$num_dam
km_dam <- py$km_dam
pct_dam <- py$pct_dam
resource_km <- py$resource_km
resource_pct <- py$resource_pct
demo_km <- py$demo_km
demo_pct <- py$demo_pct
resource_sev <- py$resource_sev
demo_sev <- py$demo_sev
sum_road <- py$sum_road

```




There are `r num_dam` mapped small dams on “potentially accessible” stream segments in the watershed, blocking a total of `r km_dam` km (~`r pct_dam`% of the total habitat) of modelled spawning and rearing habitat for anadromous salmon, resulting in a medium extent. The extent rating of these structures was confirmed by the planning team. There are two known fish-passage structures in the watershed, including on the dam at the outlet of McKinley Lake. The remaining dams likely block passage for anadromous salmon and would require significant resources to remediate. However, due to the limited extent of dams in the watershed, a final pressure rating of Medium was assigned. Four small dams were identified on the priority barrier list (see Appendix B). Three of the dams require further assessment and confirmation of upstream habitat quality, and the dam observed at the outlet of Kwun Lake does not exist. 

### Road-stream Crossings {-}

Road-stream crossings are the most abundant barrier type in the watershed, with `r sum_road` assessed and modelled crossings located on stream segments with modelled habitat. Demographic road crossings (highways, municipal, and paved roads) block `r demo_km` km of habitat (~`r demo_pct`% of the total blocked habitat), with `r demo_sev`% of assessed crossings having been identified as barriers to fish passage. Resource roads block `r resource_km` km of habitat (~`r resource_pct`%), with `r resource_sev`% of assessed crossings having been identified as barriers. The planning team felt that the data was underestimating the severity of road-stream crossing barriers in the watershed, and therefore decided to update the rating from High to Very High. The planning team also felt that an irreversibility rating of Medium was appropriate due to the technical complexity and resources required to remediate road-stream crossings.

### Trail-stream crossings{-}

There is very little spatial data available on trail-stream crossings in the watershed, so the planning team was unable to quantify the true Extent and Severity of this barrier type. However, the planning team felt that trail-stream crossings are not prevalent within the watershed and that, where they do exist, they do not significantly impact passage for anadromous salmon. As most crossings will be fords or similar structures, remediation may not be required, or remediation costs associated with these barriers would be quite low. Overall, the planning team felt that the pressure rating for trail-stream crossings was likely Low; however, the lack of ground-truthed evidence to support this rating was identified as a knowledge gap within this plan. 

### Lateral Barriers{-}

There are numerous types of lateral barriers that potentially occur in the watershed, including dykes, berms, and linear development (i.e., road and rail lines), all of which can restrict the ability of anadromous salmon to move into floodplains, riparian wetlands, and other off-channel habitats. No comprehensive lateral barrier data exists within the watershed, so pressure ratings were based on qualitative local knowledge. Lateral barriers are not thought to be as prevalent as road- or rail-stream crossings but are likely very severe where they do exist. Significant lateral barriers are known to occur along the mainstem of the Horsefly River, which disconnect the mainstem river from historic floodplain and off-channel habitat. Overall, the planning team decided that a High pressure rating adequately captured the effect that lateral barriers are having on connectivity in the watershed. Work to begin quantifying and mapping lateral habitat will begin in 2022-23, as described in the Operational Plan under Strategy 2: Lateral barrier remediation.  

### Natural Barriers {-}

Natural barriers to fish passage can include debris flows, log jams, sediment deposits, etc., but natural features that have always restricted fish passage (e.g., waterfalls) are not considered under this barrier type. Natural barriers are difficult to include in a spatial prioritization framework due to their transient nature. The planning team identified known natural barriers that occur throughout the watershed, such as beaver dams and log jams. Generally, these natural barriers are only severe impediments to fish passage during low-flow years, but reduced baseflows have become more common in recent years. Based on this, the planning team felt that natural barriers will be severe most years where they exist, but are mostly reversible, resulting in an overall pressure rating of Low. 

## Situation Analysis {-}

The following situation model was developed by the WCRP planning team to “map” the project context and brainstorm potential actions for implementation. Green text is used to identify actions that were selected for implementation (see Strategies & Actions), and red text is used to identify actions that the project team has decided to exclude from the current iteration of the plan, as they were either outside of the project scope, or were deemed to be ineffective by the planning team. 

![Situation analysis developed by the planning team to identify factors that contribute to fragmentation (orange boxes), biophysical results (brown boxes), and potential strategies/actions to improve connectivity (yellow hexagons) for target species in the Horsefly River watershed.](images/figure3.png){#fig-sitan}

## Goals {-}


In [6]:
#| label: tbl-goals
#| tbl-cap: 'Goals to improve (1) spawning and rearing and (2) overwintering habitat connectivity for target species in the Horsefly River watershed over the lifespan of the WCRP (2021-2040). The goals were established through discussions with the planning team and represent the resulting desired state of connectivity in the watershed. The goals are subject to change as more information and data are collected over the course of the plan timeline (e.g., the current connectivity status is updated based on barrier field assessments).'
#| warning: false
#| echo: false

#creating table 7
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

df = pd.DataFrame({"Goal #": [1,2],
                   "Goal": ["By 2040, the percent (%) of total linear habitat accessible to anadromous salmon will increase from 94% to 96% within the Horsefly River watershed (i.e., reconnect at least 11.7 km of habitat).",
                            "By 2024, the total area of overwintering habitat accessible to Anadromous Salmon will increase by 1,500 m2 within the Horsefly River watershed. "]
                    })

data = df.style.hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }])

Goal #,Goal
1,"By 2040, the percent (%) of total linear habitat accessible to anadromous salmon will increase from 94% to 96% within the Horsefly River watershed (i.e., reconnect at least 11.7 km of habitat)."
2,"By 2024, the total area of overwintering habitat accessible to Anadromous Salmon will increase by 1,500 m2 within the Horsefly River watershed."


## Strategies & Actions {-}

Effectiveness evaluation of identified conservation strategies and associated actions to improve connectivity for target species in the Horsefly River watershed. The planning team identified five broad strategies to implement through this WCRP, 1) crossing remediation, 2) lateral barrier remediation, 3) dam remediation, 4) barrier prevention, and 5) communication and education. Individual actions were qualitatively evaluated based on the anticipated effect each action will have on realizing on-the-ground gains in connectivity. Effectiveness ratings are based on a combination of "Feasibility and "Impact", Feasibility is defined as the degree to which the project team can implement the action within realistic constraints (financial, time, ethical, etc.) and Impact is the degree to which the action is likely to contribute to achieving one or more of the goals established in this plan.

## Strategy 1: Crossing Remediation {-}


In [7]:
#| label: tbl-S1
#| tbl-cap: Strategy 1
#| warning: false
#| echo: false

import numpy as np
from IPython.display import display
import pandas as pd

data = pd.read_csv('data/Strategy1.csv', index_col=False, skip_blank_lines=False )
 
def fix_table(val):
    return str(val)

def highlighttab7(val):
    red = '#ff0000;'
    yellow = '#ffff00;'
    lgreen = '#92d050;'
    dgreen = '#03853e;'

    if val=="Medium" or val=="Need more information": color = yellow
    elif val=="Very high" or val=="Very effective" : color = lgreen
    elif val =="High" or val=="Effective": color = dgreen
    else: color = 'white'
    return 'background-color: %s' % color

data = data.replace(np.nan, '', regex=True)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})
data = data.set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }])

## Strategy 2: Lateral Barrier Remediation {-}


In [8]:
#| label: tbl-S2
#| tbl-cap: Strategy 2
#| warning: false
#| echo: false

data = pd.read_csv('data/Strategy2.csv', escapechar='\n', index_col=False)

data = data.replace(np.nan, '', regex=True)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }])

ID,Actions,Details,Feasibility,Impact,Effectiveness
2.1,Remediate dikes / berms / other lateral barriers,,High,Very high,Effective
2.2,Initiate a barrier owner outreach program,,Very high,Very high,Very effective
2.3,"Knowledge Gap: Identify and map year-round lateral habitat, as well as overwintering habitat","Explore the use of a drone to identify lateral habitat. - Volunteers from the HRR will conduct field habitat assessments following modules in the Pacific Streamkeepers Handbook to assess disconnected lateral and overwintering salmon habitats in the Horsefly watershed CNFASAR proposal: -Funding for equipment in 2022-2023, and for field transportation in 2022-2023, 2023-2024",Very high,Very high,Very effective
2.4,Knowledge Gap: Map lateral barriers and barrier ownership,Focus on identifying ownership of priority lateral barriers that we want to remediate in the short-term.,Very high,Very high,Very effective
2.5,Knowledge Gap: Develop a framework to assess and prioritize between different lateral barrier remediation projects,"CWF is leading a provincial-scale analysis of the effect of rail lines on connectivity for Anadromous Salmonids, as part of this project lateral habitat and barrier assessments and prioritization methods will be developed.",Very high,Very high,Very effective


## Strategy 3: Dam Remediation {-}


In [9]:
#| label: tbl-S3
#| tbl-cap: Strategy 3
#| warning: false
#| echo: false

data = pd.read_csv('data/Strategy3.csv', index_col=False)

data = data.replace(np.nan, '', regex=True)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }])

ID,Actions,Details,Feasibility,Impact,Effectiveness
3.1,Remediate Dams,,Medium,Very high,Need more information
3.2,Install Fish Passage,,Medium,High,Need more information
3.3,Connect with Cattleman's Association to explore a partnership to remediate dams,This may involve exploring alternative water management actions that would allow for the remediation of irrigation dams.,High,Medium,Need more information
3.4,Knowledge Gap: Continue updating the barrier prioritization model,The model has been updated to reflect 2021 field assessments and intermediate barrier review.,Very high,High,Effective
3.5,Knowledge Gap: Assess dams to determine whether they exist and are truly blocking fish habitat,Four dams were assessed during 2021 field season; additional field assessment needed.,Very high,High,Effective
3.6,Knowledge Gap: Identify and map dam ownership,,Very high,Very high,Very effective


## Strategy 4: Barrier Prevention {-}


In [10]:
#| label: tbl-S4
#| tbl-cap: Strategy 4
#| warning: false
#| echo: false

import pandas as pd

data = pd.read_csv('data/Strategy4.csv', index_col=False)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }])

ID,Actions,Details,Feasibility,Impact,Effectiveness
4.1,Explore potential partnerships with industrial companies,Invite industrial players to a workshop on how to apply crossing / lateral barrier BMPs. BMPs could include those that minimize the need for road-stream crossings.,Very high,High,Effective
4.2,Stabilize sediment sources that are explicitly linked to sediment wedges or erosion that are acting as barriers,"This could include numerous bank stabilization techniques, including restoring riparian vegetation. This applies to some tributaries that have altered confluence areas - the link needs to be made between confluence alterations and timing of movement for juvenile fish. Local ranchers and Cattleman's association could be engaged, as well as forestry licensees.",Very high,Medium,Need more information


## Strategy 5: Communication and Education {-}


In [11]:
#| label: tbl-S5
#| tbl-cap: Strategy 5
#| warning: false
#| echo: false

data = pd.read_csv('data/Strategy5.csv', index_col=False)

data = data.replace(np.nan, '', regex=True)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }])

ID,Actions,Details
5.1,Implement the WCRP Progress Tracking Plan,The WCRP Progress Tracking Plan will help the team determine if we are achieving our goals and objectives.
5.2,Develop a communication strategy to raise awareness and support for this WCRP,"This intervention includes communicating both the WCRP and the collaborative process in developing it, as well as communicating outcomes (e.g., barrier remediations). CNFASAR proposal: - HRR will work with CWF to develop outreach and communications materials, including press releases, social media content, a video, and content for their website - With HRR, CWF will present on fish passage issues and solutions at the annual Horsefly River Salmon Festival"


## Theories of Change & Objectives {-}

Theories of Change are explicit assumptions around how the identified actions will achieve gains in connectivity and contribute towards reaching the goals of the plan. To develop Theories of Change, the planning team developed explicit assumptions for each strategy which helped to clarify the rationale used for undertaking actions and provided an opportunity for feedback on invalid assumptions or missing opportunities. The Theories of Change are results oriented and clearly define the expected outcome. The following theory of change models were developed by the WCRP planning team to “map” the causal (“if-then”) progression of assumptions of how the actions within a strategy work together to achieve project goals. 

![Theory of change developed by the planning team for the actions identified under Strategy 1: Crossing Remediation in the Horsefly River watershed.](images/figure4.png){#fig-stra1}

![Theory of change developed by the planning team for the actions identified under Strategy 2: Lateral Barrier Remediation in the Horsefly River watershed.](images/figure4.png){#fig-stra2}

![Theory of change developed by the planning team for the actions identified under Strategy 3: Dam Remediation in the Horsefly River watershed.](images/figure6.png){#fig-stra3}

![Theory of change developed by the planning team for the actions identified under Strategy 4: Barrier Prevention in the Horsefly River watershed.](images/figure7.png){#fig-stra4}

## Operational Plan {-}

The operational plan represents a preliminary exercise undertaken by the planning team to identify the potential leads, potential participants, and estimated cost for the implementation of each action in the Horsefly River watershed. The table below summarizes individuals, groups, or organizations that the planning team felt could lead or participate in the implementation of the plan and should be interpreted as the first step in on-going planning and engagement to develop more detailed and sophisticated action plans for each entry in the table. The individuals, groups, and organizations listed under the "Lead(s)" or "Potential Participants" columns are those that provisionally expressed interest in participating in one of those roles or were suggested by the planning team for further engagement (denoted in bold), for those that are not members of the planning team. The leads, participants, and estimated costs in the operational plan are not binding nor an official commitment of resources, but rather provide a roadmap for future coordination and engagement to work towards implementation of the WCRP.


In [12]:
#| label: tbl-opplan
#| tbl-cap: Operational plan to support the implementation of strategies and actions to improve connectivity for target species in the Horsefly River watershed.
#| warning: false
#| echo: false

from IPython.display import display
import pandas as pd
import numpy as np

def df_operation(val):
    return "background-color: black; color: white"


data = pd.read_csv('data/Table13.csv', index_col=False)

data = data.replace(np.nan, '', regex=True)

rows = pd.IndexSlice[[0,10,16,23,26,29,30,31], :]

data = data.style.applymap(df_operation, subset=rows).hide_index().set_properties(**{'text-align': 'left'})
data.set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', '#008270'),('text-align', 'left')]
   }])

Strategy / Actions,Lead(s) [1],Participants3,Total Budget
Strategy 1: Crossing Remediation,,,"$3,666,300.00"
1.1 – Remediate crossings that are acting as barriers,CWF,"Horsefly River Roundtable, Fisheries and Oceans Canada (DFO)","$3,500,000.00"
1.2 – Lobby that the government enforce their regulations,TBD,"CWF, Horsefly River Roundtable, Williams Lake First Nation (WLFN)","$10,000.00"
1.3 – Initiate a barrier owner outreach program for locations on the barrier remediation shortlist,"HRR, CWF, DFO",,TBD
1.4 – Knowledge Gap: Continue updating the barrier prioritization model,CWF,TBD,"$100,000.00"
1.5 – Knowledge Gap: conduct field assessments on updated preliminary barrier list using the provincial fish passage framework and update connectivity goal if additional barriers are added to the barrier remediation shortlist,CWF,"Horsefly River Roundtable, DFO","$50,300.00"
1.6 - Update longitudinal connectivity goal if additional barriers are added to the barrier remediation shortlist,,,
1.7 – Knowledge Gap: Identify and map crossing ownership for barriers on the barrier remediation shortlist,TBD,"CWF, DFO (Anthonie)","$1,500.00"
1.8 – Knowledge Gap: Compile road maintenance schedules,DFO,"CWF, WLFN, DFO, FLNRORD","$2,000.00"
1.9 – Knowledge Gap: Survey trail-stream crossings to confirm low pressure rating values,WLFN,"CWF, DFO","$2,500.00"


## Funding Sources {-}



```{r fund, echo = FALSE, results = 'asis'}
#| label: tbl-fund
#| tbl-cap: "Potential funding sources for plan implementation in the Horsefly River watershed. The Canadian Wildlife Federation and the planning team can coordinate proposal submission through these sources."
#| warning: false

library("flextable")

data <- read.csv("data/Table14.csv", check.names=FALSE)
ft <- flextable(data)
ft <- bg(ft, bg = "#008270", part = "header")
ft <- color(ft, color = "white", part = "header")
ft <- set_caption(ft)
ft |> autofit() 
```